In [ ]:
-- Welcome to your new notebook
-- Type here in the cell editor to add code!--https://www.youtube.com/watch?v=7kjrRveFNro

In [2]:
%%sql
CREATE TABLE IF NOT EXISTS customers_dim(customerID int,name string, address string, current boolean, effectiveDate timestamp,endDate timestamp )

StatementMeta(, c3faca5d-63ee-4339-b5d8-dd384aba50bc, 3, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [3]:
insert into customers_dim values 
(1,'Mahesh','Bangalore',1,CURRENT_TIMESTAMP(),'9999-12-31'),
(2,'ram','Hyderabad',1,CURRENT_TIMESTAMP(),'9999-12-31'),
(3,'ravy','Chennai',1,CURRENT_TIMESTAMP(),'9999-12-31'),
(4,'raj','Pune',1,CURRENT_TIMESTAMP(),'9999-12-31')

StatementMeta(, c3faca5d-63ee-4339-b5d8-dd384aba50bc, 4, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [4]:
select * from customers_dim

StatementMeta(, c3faca5d-63ee-4339-b5d8-dd384aba50bc, 5, Finished, Available)

<Spark SQL result set with 4 rows and 6 fields>

In [5]:
%%sql
CREATE TABLE IF NOT EXISTS updates_source(customerID int,name string, address string, effectiveDate timestamp)

StatementMeta(, c3faca5d-63ee-4339-b5d8-dd384aba50bc, 6, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [7]:
insert into updates_source values
(5,'Sridhar','Delhi',CURRENT_TIMESTAMP()),
(6,'Prasad','Mumbai',CURRENT_TIMESTAMP()),
(2,'ram','Bangalore',CURRENT_TIMESTAMP()),
(1,'Mahesh','Hyderabad',CURRENT_TIMESTAMP())

StatementMeta(, c3faca5d-63ee-4339-b5d8-dd384aba50bc, 8, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [9]:
select * from updates_source

StatementMeta(, c3faca5d-63ee-4339-b5d8-dd384aba50bc, 10, Finished, Available)

<Spark SQL result set with 4 rows and 4 fields>

#### Merge 2 bảng dim và updates source để tracking những data đã được thay đổi, merge key của 2 cột thay đổi là null ####

In [12]:
SELECT updates_source.customerID as mergeKey, updates_source.*
FROM updates_source
UNION ALL
SELECT NULL as mergeKey,updates_source.*
FROM updates_source JOIN customers_dim ON updates_source.customerID = customers_dim.customerID
WHERE customers_dim.current = 1 AND customers_dim.address <> updates_source.address

StatementMeta(, c3faca5d-63ee-4339-b5d8-dd384aba50bc, 13, Finished, Available)

<Spark SQL result set with 6 rows and 5 fields>

#### Updates các giá trị vào bảng customers_dim ####
- Nếu bảng updates có những record giống như data bảng dim thì set lại data bảng dim true thành false và endDate = ngày effectiveDate (bảng Union mới và cũ)
- Nếu ko giống thì insert các giá trị từ giá trị của bảng Union All

In [18]:
MERGE INTO customers_dim as customers
USING (
    SELECT updates_source.customerID as mergeKey, updates_source.*
    FROM updates_source
    UNION ALL
    SELECT NULL as mergeKey, updates_source.*
    FROM updates_source JOIN customers_dim as customers ON updates_source.customerID = customers.customerID
    WHERE customers.current = 1 AND customers.address <> updates_source.address
) staged_updates ON customers.customerID = staged_updates.mergeKey
WHEN MATCHED AND customers.current = 1 AND customers.address <> staged_updates.address 
THEN
    UPDATE SET current = 0, endDate = staged_updates.effectiveDate 
WHEN NOT MATCHED 
THEN
    INSERT (customerID, name, address, current, effectiveDate, endDate)
    VALUES (staged_updates.customerID, staged_updates.name, staged_updates.address, 1, staged_updates.effectiveDate, '9999-12-31');

StatementMeta(, c3faca5d-63ee-4339-b5d8-dd384aba50bc, 19, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>

In [19]:
select * from customers_dim

StatementMeta(, c3faca5d-63ee-4339-b5d8-dd384aba50bc, 20, Finished, Available)

<Spark SQL result set with 8 rows and 6 fields>